# **ICT303 - Advanced Machine Learning and Artificial Intelligence**
# **Lab 3 - Linear Neural Networks**

The goal of this lab is to learn how to implement, from scratch, the linear regression that we covered in the lecture. While deep learning (and other) programming frameworks already provide ready-to-use implementations, implementing things from scratch is the only way to make sure that you
- understand the theoretical concepts,
- really know what you are doing,  
- can use efficiently these tools,
- understand their limitations, and
- can extend and build on the top of them to solve complex problems.

This week, we will first start with a naive approach that implements the analytical solution of the linear regression model and run it on synthetically-generated data. For this, we will introduce the class **SyntheticRegressionData**.

We will then extend the implementation by introducing the classes **LinearRegression** and **Trainer**, which implement the standard machine learning pipeline. This will give you an idea of what is involved in the implementation of machine learning models.

Finally, you will be required to create these classes in separate modules (not just on the notebook) so that you can easily reuse them later.

This lab is adapted from: https://d2l.ai/chapter_linear-regression/linear-regression-scratch.html

### **1. A short summary of linear regression**

Recall, from the lecture, that  regression is the problem of estimating the value of a variable (output) $y$ as a function of $d$ variables (input) $\textbf{x} = (x_1, \dots, x_d)^\top$. When using linear regression, this function, which defines the relation between the input $\textbf{x}$ and the output $y$, is linear. In other words:

$$
y = \langle \textbf{W}, \textbf{x} \rangle + b = \textbf{W}^{\top} \textbf{x} + b = \sum_{i=1}^{d}w_ix_i + b,
$$

where
- $\textbf{W}$ is a column vector (thus $\textbf{W}^\top$ is a row vector), i.e., $\textbf{W} = (w_1, \cdots, w_d)^\top$.
- $b$ is a scalar and is called a bias.

We can also write $\textbf{x} = (x_1, \dots, x_d, 1)^\top$ and $\textbf{W} = (w_1, \cdots, w_d, b)^\top$ so that the equation above becomes:
$$
y = \langle \textbf{W}, \textbf{x} \rangle = \langle \textbf{x} , \textbf{W}\rangle = \textbf{W}^{\top} \textbf{x} = \textbf{x}^{\top} \textbf{W}.
$$

$(w_1, \cdots, w_d)$ and $b$ are called **parameters** of the model. The goal of the  training process is to find the right values of $(w_1, \cdots, w_d)$ and $b$ from some training data (sometimes also called exemplars).

Suppose that we have the following training data:
-  $n$ input samples $\textbf{x}^1, \dots, \textbf{x}^n$ stacked into a matrix $\textbf{X}$ (the $i-$th column of the matrix $\textbf{X}$ is actually the column vector $\textbf{x}^i$).
- For each sample $\textbf{x}^i$, we have its desired output $y^i$. Let's put all the $y^i$'s into one column vector $\textbf{y}$.

We have seen during the lecture that the best values of the parameters $(w_1, \cdots, w_d)$  and $b$ are those that minimize the following **loss function**:

$$
\mathcal{l}(\textbf{X}, \textbf{y}, \textbf{W}) = \frac{1}{n}\|\textbf{y} - (\textbf{W}^\top \textbf{X}) \|^2.
$$

For further details, please refer to the lecture slides and the lecture recording.

### **2. Generating synthetic data**

Machine learning is all about data. The data usually comes from practical problems. However, when developing machine learning models, we usually rely on synthetic data because:
- They are easy to generate, by just writing some code
- We know the input, the output, and  the process that generated them. Thus, we can easily evaluate whether our machine learning model works the way it is intended to.

In the case of linear regression, we will need training data in the form of pairs $(\textbf{x}^i, y^i)$ where $\textbf{x}^i$ is the input and $y^i$ its corresponding output (called also label). In this example, let's start with a simple case where $\textbf{x}^i$ is a scalar (i.e., just one value).

Let's simulate $1000$ data points (i.e., $n=1000$). We generate each $\textbf{x}^i$ randomly from a normal (Gaussian) distribution with mean $0$ and standard deviation $\sigma = 0.01$ (any other values are also fine). Then, its corresponding output $y^i$ is  given by:
\begin{equation}
      y^i = \textbf{w}^\top \textbf{x}^i + b + \varepsilon.
\end{equation}

Here, $\varepsilon$ is an additive **noise** that corrupts the data generation process. (Think of a device that captures data where, during the capture process, the data gets corrupted with some noise.)

In this example, let's set $\textbf{w} = [2]$ and $b=4.2$.

Let's create a class called **SyntheticRegressionData** whose responsibiity is to generate synthetic data following the description above:

In [2]:
import torch

class SyntheticRegressionData:
  def __init__(self, w, b, noise=0.01, num_train=1000):
    self.n = num_train   # no. of trianing samples
    self.w = w
    self.b = b

    # generate self.n samples. Each sample is of dimension length of self.w
    self.X = torch.randn(self.n, len(self.w))

    # Get a random noise
    noise = torch.randn(self.n, 1) * noise

    # For each sample in X, generate its corresponding y value
    # using the equation above.
    # Note below that the function reshape has parameters (-1, 1).
    # This means that it will reshape the vector w into a 2D matrix where:
    # - the 2nd dimension has length 1
    # - the length of the first dimension will be autmatically set.
    #
    self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise


Now let's test this class by generating $n=1000$ data and plot them.

In [ ]:
w = torch.tensor([2.0])
b = 4.2
num_train = 1000
noise = 1.2
data = SyntheticRegressionData(w, b, noise, num_train)

# Let's print some data samples
print(data.X[0], data.y[0])

Often, we need to **plot** or **visualize** the data instead of printing individual numerical values. Python offers a range of tools for plotting data and graphs. For instance, in the simple example above, we can use the **plot** function from **matloplib**:

In [ ]:
import matplotlib.pyplot as plt

# Plot as points in green
plt.plot(data.X, data.y, 'r.')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Random data')

**Questions:**


1. Change the values of noise (make it smaller or bigger) and observe the plot that is generated.



In [ ]:
## Write the code here and test it

2. Can the class above be used to generate high dimensionial data, e.g., 2D or 3D points, or even points in nD? Write below the Python code that would do that.

In [4]:
## Write the code here and test it

w2 = torch.tensor([2.0, 3.0])
b2 = 2.5
num_train = 1000
noise = 1.2
data3 = SyntheticRegressionData(w2, b2, noise, num_train)

print(data3.X.shape, data3.y.shape)

torch.Size([1000, 2]) torch.Size([1000, 1])


### **3. Analytical solution to the linear regression problem**

Since this is a linear model, the solution has an analytical form. To simplify things, we add the bias $b$ to the vector $\textbf{W}$. Thus, in what follows, $\textbf{W} = (w_1, \dots, w_d, b)^\top$. We also add the element $1$ to each vector $\textbf{x}^i$. With this notation, the solution of the equation above, i.e., the best values of the parameters $\textbf{W}$, is given by:

$$
\textbf{W}^*  = (\textbf{X} \textbf{X}^\top)^{-1} \textbf{X} \textbf{y}.
$$

In the equation above:
- $\textbf{X}$ is a $(d+1) \times n$ matrix, where each column is a training sample with the value $1$ appended to it. $n$ is the number of samples we generated
- $\textbf{X}^\top$ means the transpose of the matrix $\textbf{X}$.
- The $-1$ exponent means the inverse of a matrix.

#### **Exercise 1 - Naive implementation**

Using PyTorch, let's implement the analytical solution for the linear regression problem. We will then use the synthetic data generator we created above to test our solution.

In [8]:
# Step 1 - Generate some synthetic data
w = torch.tensor([2.0])   # Note that the length of W is the dimension of the data
b = 4.2
num_train = 5
noise = 0.01
data = SyntheticRegressionData(w, b, noise, num_train)

# Step 2 - linear regression
# From this data, we need to recover w and b - let's call them w_star and b_star
# Ideally, if our model is accurate, w_start and b_star should be equal to w and b, respectively
# The they are to w and b, the better is our solution

# 2.1. Add 1 at the end of X
ones = torch.ones(data.n, 1)     # a column vector of ones
X = torch.cat((data.X, ones), 1) # Append the columns of ones to the end of data.X. So I need to transpose it
X = torch.transpose(X, 0, 1)

# 2.2. The solution to the linear regression
# print(data.X[0], data.y[0])
A = torch.matmul(X, torch.transpose(X, 0, 1))
# A should be of size d x d. To check it, uncomment the following
# print(A.size())


B = torch.matmul(torch.inverse(A), X)
# print(B.size())

w_estimated = torch.matmul(B, data.y)
w_star = w_estimated[0:-1]    # get all the elements except the last one
b_star = w_estimated[-1:]     # last element

# print(w_estimated)
print("Estimated W: ", w_star)
print("Estimate b: ", b_star)

# real values
print("Real W: ", w)
print("Real b: ", b)


Estimated W:  tensor([[1.9956]])
Estimate b:  tensor([[4.1967]])
Real W:  tensor([2.])
Real b:  4.2


**Question:** In the code above, try the followings:
- Set the variable **noise** to $0$, and observe the results produced by the code.
- Increase the amount of noise in the generated synthetic data, i.e., by setting the variable **noise** to larger values, and observe the results produced by the code.

What can you say about the accuracy?

#### **Exercise 2 - Analysis of the solution**

What do you think would be the issues with the solution above? Think of:
- The computation time: think first of what would be the issue and then test the code. You can add some code lines to check the computation time it takes to find te parameters $\textbf{w}$ and $b$,  
- The computation of the inverse of the matrix $\textbf{X} \textbf{X}^\top$. Does the inverse always exist? If no, in which conditions it would not exist?

#### **Exercise 3 - Practical problem**

 We would like to write a code that predicts house prices based on their land **area** and **age**. We assume that the house price is a linear function of **area** and **age**.

Use the classes you created above to solve this problem. To test your solution, use the synthetic data generator class to simulate some data. For example, you can simulate data by calling the class **SyntheticRegressionData** with the parameters $w_1 = 100, w_2 = -5, b=100$.

Make sure you make the necessary changes to the solution above in order to solve this problem. However, when you make changes, make sure that the classes also work for the previous problem.

In [18]:
## Create the necessary code here

# Price of a house is in a linear relationship to two of it's attributes, area and age.
# So, I'll take w1 = area and w2 = age.
# And now I need to create the synthetic data for this

whouse = torch.tensor([100.0, -5.0]) #a tensor with 2 elements, so two features of the house
bhouse = 100
num_train = 1000
noise = 1.2
datahouse = SyntheticRegressionData(whouse, bhouse, noise, num_train)

#print(datahouse.X.shape, datahouse.y.shape)
# Expected output is torch.Size([1000, 2]) torch.Size([1000, 1])
# At this point we have successfully created the synthetic data

# Now, we use the analytical solution to get the w and b values that we set.
# We can follow the example above

# Add the ones to the end of the matrix X
ones = torch.ones(datahouse.n, 1)
X = torch.cat((datahouse.X, ones), 1)
X = torch.transpose(X, 0, 1)

# 2.2. The solution to the linear regression
#print(data.X[0], data.y[0])
A = torch.matmul(X, torch.transpose(X, 0, 1))
# A should be of size d x d. To check it, uncomment the following
#print(A.size())

B = torch.matmul(torch.inverse(A), X)
# print(B.size())

w_estimated = torch.matmul(B, datahouse.y)
w_star = w_estimated[0:-1]    # get all the elements except the last one
b_star = w_estimated[-1:]     # last element

# print(w_estimated)
print("Estimated W: ", w_star)
print("Estimate b: ", b_star)

# real values
print("Real W: ", whouse)
print("Real b: ", bhouse)

Estimated W:  tensor([[100.0191],
        [ -5.0274]])
Estimate b:  tensor([[100.0510]])
Real W:  tensor([100.,  -5.])
Real b:  100


#### **Exercise 4 - Proper implementation**

In the code above, we implemented the synthetic data generator as a class but the linear regression solver as part of the main code. In this exercise, let's also implement the solver as a class.

To do so, we will create a class called LinearRegression. It will have at least the following parameters and methods as members:
- The weights $\textbf{W}$ and bias $b$. When the class is created, these weights will be initialized to random values. Note that, in the current solution, these weights can also be initialized to $0$. We will show later when using the general solution that initializing to $0$ is not a good idea.
- The method **forward**, which takes an input $\textbf{X}$ and computes its output $y$.


In [ ]:
class LinearRegression:
  def __init__(self, d=1):

    self.dim = d  # dimension of the data

    # Set the weights to random values from normal distribution of mean 0 and std 0.01
    self.w = torch.randn(self.dim, 1) * 0.01
    self.b = torch.randn(1, 1) * 0.01         # the bias

  # The forward function, which computes y for a given X
  def forward(self, X):
    y = torch.matmul(self.X, w.reshape((-1, 1))) + b
    return y



Now we need to define the training class, i.e., the class that takes the generated data, and then it fits to it the LinearRegression model (i.e, estimates the optimal parameters of the linear regression). Let's call it **Trainer**, which has at least the following members:
- Method **fit()**, which takes as input the model (LinearRegression in our case) and the data (the synthetic data we generated in our case), and computes the optimal parameters **$\textbf{W}$** and **$b$** of the model.

In [ ]:
class Trainer:
  def __init__(self):
    return

  def fit(self, model, data):
    # 2.1. Add 1 at the end of X
    ones = torch.ones(data.n, 1)  # a column vector of ones
    X = torch.cat((data.X, ones), 1)

    # 2.2. The solution

    # print(data.X[0], data.y[0])
    A = torch.matmul(torch.transpose(X, 0, 1),X)
    # A should be of size num_train x num_train. To check it, uncomment the following
    # A.size()

    B = torch.matmul(torch.inverse(A), torch.transpose(X, 0, 1))
    # B.size()

    w_estimated = torch.matmul(B, data.y)
    model.w = w_estimated[0:-1]    # get all the elements except the last one
    model.b = w_estimated[-1:]     # last element


Now, let's write the code of the main function:

In [ ]:
# 1. The data
d = 1   # dimension of the data
w = torch.tensor([2.0])
b = 4.2
num_train = 1000
noise = 0.01
data = SyntheticRegressionData(w, b, noise, num_train)

# 2. The model (linear regression)
model = LinearRegression(d)

# 3. The trainer
trainer = Trainer()

# 4. Estimate the parameters
trainer.fit(model, data)

# 5. Get the results
w_star = model.w
b_star = model.b

# print(w_estimated)
print("Estimated W: ", w_star)
print("Estimate b: ",  b_star)

# real values
print("Real W: ", w)
print("Real b: ", b)

Test the code above on multidimensional data (e.g., 2D or 3D).

#### **Exercise 5 - Another problem**

Suppose now that the problem we would like to solve is:
$$
y = \sum_{i=1}^{d}w_ix_i^2 + b,
$$
where $x_i \in \mathbb{R}$, i.e., $x_i$'s are one-dimensinal scalars. How would you extend the implementation above to solve this problem? Can you think of a better Object-Oriented Design that will make our implementation generic, so that it can be extended with minimum efforts to other machine learning models?

## **4. Reusable implementation**

In the examples above, we wrote all the code in the google colab notebook. Thus, the classes and methods we created cannot be reused elsewhere. In this exercise, create your first library of classes saved into python files (.py), and write here the test code that uses them.